In [17]:
import pandas as pd
import numpy as np
import random
import time

# Types of evaluations
- Percentages:
    - Exam: 30 - 50%
    - Assignments: 10% each
    - Essays: 10% each
    - Projects: 30 - 50%
- Rules:
    - Exam: yes/no
    - Project: yes/no
    - Remaining %: assignments or essays

In [18]:
def get_cutoff(grade):
    if grade >= 20 and grade <= 49:
        return (20, 49)
    if grade >= 50 and grade <= 59:
        return (50, 59)
    if grade >= 60 and grade <= 69:
        return (60, 69)
    if grade >= 70 and grade <= 79:
        return (70, 79)
    if grade >= 80 and grade <= 89:
        return (80, 89)
    if grade >= 90 and grade <= 100:
        return (90, 100)

In [19]:
exam_perc = [30, 40, 50]
proj_perc = [30, 40, 50]
cutoffs_dict = {
    'A': (90, 100),
    'B+': (80, 89),
    'B': (70, 79),
    'C+': (60, 69),
    'C': (50, 59),
    'F': (20, 49)
}
grades = ['A', 'B+', 'B', 'C+', 'C', 'F']

def assign_eval(row):
    grade = row['grade']
    perc_sum = 0
    avg = 0
    row_objs = []
    if grade != -1:
        # Courses taken
        cutoff = get_cutoff(grade)
        exam = random.random()
        if exam >= 0.5:
            perc = random.choice(exam_perc)
            perc_sum += perc
            score = random.randint(cutoff[0], cutoff[1])
            avg += score * perc / 100
            row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': 'exam', 'weight': perc, 'score': score}
            row_objs.append(row_obj)
        proj = random.random()
        if proj >= 0.5:
            perc = random.choice(proj_perc)
            perc_sum += perc
            if perc_sum != 100: # Not the last one
                score = random.randint(cutoff[0], cutoff[1])
                avg += score * perc / 100
            else: # Is the last one
                score = max(0, np.round((grade - avg) / (perc / 100), decimals=1))
            row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': 'project', 'weight': perc, 'score': score}
            row_objs.append(row_obj)
        if perc_sum < 100:
            perc_remain = 100 - perc_sum
            evals_remain = []
            essay_assign = random.random()
            for i in range(int(perc_remain / 10)):
                if essay_assign >= 0.5:
                    evals_remain.append('essay')
                else:
                    evals_remain.append('assignment')
            for i in range(len(evals_remain) - 1):
                score = random.randint(cutoff[0], cutoff[1])
                avg += score * 10 / 100
                row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': evals_remain[i], 'weight': 10, 'score': score}
                row_objs.append(row_obj)
            # Last one
            score = max(0, np.round((grade - avg) / 0.1, decimals=1))
            row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': evals_remain[-1], 'weight': 10, 'score': score}
            row_objs.append(row_obj)
    else:
        new_grade = random.choice(grades)
        cutoff = cutoffs_dict[new_grade]
        # Courses taking
        exam = random.random()
        if exam >= 0.5:
            perc = random.choice(exam_perc)
            perc_sum += perc
            row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': 'exam', 'weight': perc, 'score': None}
            row_objs.append(row_obj)
        proj = random.random()
        if proj >= 0.5:
            perc = random.choice(proj_perc)
            perc_sum += perc
            row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': 'project', 'weight': perc, 'score': None}
        if perc_sum < 100:
            perc_remain = 100 - perc_sum
            essay_assign = random.random()
            for i in range(int(perc_remain / 10)):
                if essay_assign >= 0.5:
                    type = 'essay'
                else:
                    type = 'assignment'
                done = random.random()
                if done >= 0.5:
                    score = random.randint(cutoff[0], cutoff[1])
                    row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': type, 'weight': 10, 'score': score}
                else:
                    row_obj = {'student_id': row['student_id'], 'course_code': row['course_code'], 'evaluation': type, 'weight': 10, 'score': None}
                row_objs.append(row_obj)
    return row_objs

In [20]:
PATH = '/Users/ngochoang/Library/CloudStorage/GoogleDrive-nhungoc1508@gmail.com/My Drive/Graduate/Semester 2 (Spring 2025)/Big Data Management/Project/P1 - Landing zone/Data/processed'

In [21]:
enrollment = pd.read_csv(f'{PATH}/enrollment.csv')
enrollment['grade'] = enrollment['grade'].fillna(-1)
print(enrollment.shape)
enrollment.head()

(222720, 5)


,student_id,course_code,semester,is_taking,grade
0,dc525,POSC 498,semester_1,0,90.0
1,dc525,POSC 377,semester_1,0,91.0
2,dc525,POSC 321,semester_1,0,90.0
3,dc525,ANTH 322,semester_1,0,90.0
4,dc525,SOCI 216,semester_1,0,98.0


In [22]:
evaluations = []
for _, enrol in enrollment.iterrows():
    rows = assign_eval(enrol)
    for row in rows:
        evaluations.append(row)

In [23]:
len(evaluations)

1537297

In [24]:
eval_df = pd.DataFrame.from_records(evaluations)
print(eval_df.shape)
eval_df.head()

(1537297, 5)


,student_id,course_code,evaluation,weight,score
0,dc525,POSC 498,exam,40,99.0
1,dc525,POSC 498,assignment,10,92.0
2,dc525,POSC 498,assignment,10,96.0
3,dc525,POSC 498,assignment,10,91.0
4,dc525,POSC 498,assignment,10,96.0


In [25]:
eval_df.to_csv(f'{PATH}/evaluations.csv', index=False)